In [15]:
!pip install gradio scikit-learn spacy --quiet
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [16]:
pip install pymupdf


In [18]:
import gradio as gr
import spacy
import fitz  # PyMuPDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Text Preprocessing
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# TF-IDF Similarity
def calculate_similarity(resume_text, jd_text):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([resume_text, jd_text])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Extract Text from .pdf or .txt
def extract_text(file_bytes, mime_type):
    if mime_type == "application/pdf":
        doc = fitz.open(stream=file_bytes, filetype="pdf")
        return "".join([page.get_text() for page in doc])
    elif mime_type == "text/plain":
        return file_bytes.decode("utf-8", errors="ignore")
    else:
        raise ValueError("⚠️ Unsupported file format. Upload only .pdf or .txt")

# Simulated Email Sending Function
def send_email(recipient_email, score, feedback, decision, cgpa):
    message = f"""
    Subject: Resume Match Results

    📈 Similarity Score: {score}
    🎓 CGPA: {cgpa}
    📋 Decision: {decision}

    Feedback:
    {feedback}

    Regards,
    AI Resume Matcher Bot
    """
    print(f"[SIMULATED EMAIL TO: {recipient_email}]\n{message}")
    return f"📨 Results simulated to be sent to {recipient_email}"

# Analysis Logic
def analyze_similarity(resume_file, jd_file, email, cgpa):
    if resume_file is None or jd_file is None:
        return 0.0, "❌ Please upload both resume and job description.", "", "", "", "", ""

    try:
        resume_mime = "application/pdf" if resume_file[:4] == b'%PDF' else "text/plain"
        jd_mime = "application/pdf" if jd_file[:4] == b'%PDF' else "text/plain"

        resume_text = extract_text(resume_file, resume_mime)
        jd_text = extract_text(jd_file, jd_mime)
    except Exception as e:
        return 0.0, f"❌ Error: {str(e)}", "", "", "", "", ""

    if not resume_text.strip() or not jd_text.strip():
        return 0.0, "❌ One or both files are empty or invalid.", "", "", "", "", ""

    resume_clean = preprocess(resume_text)
    jd_clean = preprocess(jd_text)
    score = calculate_similarity(resume_clean, jd_clean)
    cgpa_float = float(cgpa) if cgpa else 0.0

    # Decision & Feedback Logic
    if score < 0.3:
        feedback = "🔍 **Low Match**\n\nYour resume doesn't align well with the job description. Consider revising your content."
        decision = "❌ Not shortlisted. Needs major improvement."
    elif score < 0.7:
        feedback = "⚠️ **Moderate Match**\n\nYour resume shows partial alignment. Try integrating more specific job-relevant skills."
        if cgpa_float < 6.5:
            decision = "❌ Not shortlisted due to CGPA below 6.5."
        else:
            decision = "🟡 Moderate chance. Tailoring needed."
    else:
        feedback = "✅ **High Match**\n\nYour resume aligns well with the JD. Good job highlighting relevant skills!"
        if cgpa_float < 6.5:
            decision = "❌ Not shortlisted due to CGPA below 6.5."
        elif cgpa_float >= 7.5:
            decision = "🎉 Shortlisted! Excellent resume and academic record."
        else:
            decision = "✅ Shortlisted, but consider working on academic consistency."

    email_status = send_email(email, f"{score:.2f}", feedback, decision, cgpa) if email else "📭 Email not provided."

    return round(score, 2), feedback, decision, cgpa, resume_text.strip(), jd_text.strip(), email_status

# Custom Theme
custom_theme = gr.themes.Base(
    primary_hue="rose",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Poppins"), "ui-sans-serif"],
    radius_size="md",
    spacing_size="sm",
    text_size="md"
).set(
    body_background_fill="#fef9f9",
    body_text_color="#333",
    block_title_text_color="#d6336c",
    block_border_color="#f3c2d2"
)

# Gradio Interface
with gr.Blocks(theme=custom_theme) as iface:
    gr.Markdown("## ✨ AI Resume & Job Description Matcher")
    gr.Markdown("📂 Upload your **Resume** and **Job Description** in `.pdf` or `.txt` format.\n\n"
                "✉️ Enter your email to receive results.\n🎓 Add your CGPA for a better evaluation.\n\n"
                "🚀 Powered by TF-IDF + spaCy NLP.")

    with gr.Row():
        resume_input = gr.File(label="📄 Upload Resume", type="binary", file_types=[".pdf", ".txt"])
        jd_input = gr.File(label="📋 Upload Job Description", type="binary", file_types=[".pdf", ".txt"])

    with gr.Row():
        email_input = gr.Textbox(label="✉️ Email Address", placeholder="example@mail.com")
        cgpa_input = gr.Textbox(label="🎓 CGPA", placeholder="e.g., 8.23")

    analyze_btn = gr.Button("🔍 Match Resume")

    gr.Markdown("### 📊 Results")

    with gr.Row():
        score_slider = gr.Slider(label="📈 Similarity Score", minimum=0, maximum=1, step=0.01, interactive=False)
        decision_output = gr.Textbox(label="📋 Final Decision", interactive=False)

    with gr.Row():
        cgpa_output = gr.Textbox(label="🎓 Entered CGPA", interactive=False)
        email_status_output = gr.Textbox(label="📨 Email Status", interactive=False)

    feedback_output = gr.Markdown()

    with gr.Row():
        resume_output = gr.Textbox(label="📝 Extracted Resume Text", lines=10, interactive=False)
        jd_output = gr.Textbox(label="📌 Extracted JD Text", lines=10, interactive=False)

    analyze_btn.click(
        fn=analyze_similarity,
        inputs=[resume_input, jd_input, email_input, cgpa_input],
        outputs=[score_slider, feedback_output, decision_output, cgpa_output, resume_output, jd_output, email_status_output]
    )

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33165d0c127e1e0734.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
